In [1]:
import re
import pandas as pd
import PyPDF2
import os
from openpyxl import load_workbook, Workbook
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [ ]:
def getData():
    lineData = {'Date': [],
                'Transaction': [],
                'Amount': []}
    for x in os.listdir():
        if x.endswith('.pdf'):
            path = os.getcwd() + '\\' + x
            file = open(path, 'rb')
            readFile = PyPDF2.PdfFileReader(file)


            totalPages = readFile.numPages

            for i in range(totalPages):

                pageObj = readFile.getPage(i)
                pageText = pageObj.extractText

                newTrans = re.compile(r'\d{2}/\d{2}')
                moneyRe = re.compile(r'\d{1,}\.\d{2}')

                for line in pageText(pageObj).split('\n'):
                    line = re.sub(r',','',line)
                    line = re.sub(r'\$\s','',line)
                    if newTrans.match(line):

                        newValue = re.split(newTrans, line)
                        newValue = ' '.join(newValue)
                        newValue = re.split(moneyRe, newValue)
                        newValue = ' '.join(newValue)

                        newKey = newTrans.findall(line)
                        newKey = ' '.join(newKey)

                        newAmount = moneyRe.findall(line)
                        newAmount = ' '.join(newAmount)
                        

                        lineData['Date']+=[newKey]
                        lineData['Transaction']+=[newValue]
                        lineData['Amount']+=[newAmount]
        dataFrame = pd.DataFrame(lineData)
    return dataFrame

In [4]:
getData()

,Date,Transaction,Amount
0,12/22,DEPOSIT,700.00
1,12/22,ATM CHECK DEPOSIT *****30141501122,
2,12/27,eTransfer Credit Online Xfer,
3,12/29,eTransfer Credit Online Xfer,
4,01/03,ATM CASH DEPOSIT *****30141501122,
...,...,...,...
144,01/26 02/13,,74.11 39.62
145,01/27 02/14,,84.11 12.61
146,01/30 02/15,,10.38 2.58
147,02/01 02/17,,0.38 1179.00


In [5]:
def categorize():
    path = os.getcwd() + '\\' + 'Bank Data.xlsx'
    
    dataFrame = pd.DataFrame(getData())
    
    try:
        with pd.ExcelWriter('Bank Data.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
            dataFrame.to_excel(writer, sheet_name='Main')
    except:
        dataFrame.to_excel('Bank Data.xlsx', sheet_name='Main')

    category = input('What Keyword Would You Like to categorize? ')
    newCategory = dataFrame.query('Transaction.str.contains(@category)', engine='python')
    
    category = category.lower()
    newCategory = newCategory.append(dataFrame.query('Transaction.str.contains(@category)', engine='python'))
    
    category = category.upper()
    newCategory = newCategory.append(dataFrame.query('Transaction.str.contains(@category)', engine='python'))
    
    category = category.capitalize()
    newCategory = newCategory.append(dataFrame.query('Transaction.str.contains(@category)', engine='python'))
    display(newCategory)
    
    userIn = input(f'Would you like to categorize these transactions? (Y/N) ')
    userIn = userIn.lower()
    if userIn == "yes" or userIn == "y" or userIn == "yy" :
        userIn = input(f'Enter name of category: ')
        isIncome = input(f'Is this Category Income? (y/n) ')
        isIncome = isIncome.lower
        userIn=str(userIn)
        isIncome=str(isIncome)
        if isIncome == 'y' or 'yy' or 'yes':
            try:
                with pd.ExcelWriter('Bank Data Money In.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
                    newCategory.to_excel(writer, sheet_name=userIn)
                print('Done')
            except: 
                newCategory.to_excel('Bank Data Money In.xlsx', sheet_name=userIn)
                print('Done')
        else:
            try:
                with pd.ExcelWriter('Bank Data Money Out.xlsx', mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
                    newCategory.to_excel(writer, sheet_name=userIn)
                print('Done')
            except: 
                newCategory.to_excel('Bank Data Money Out.xlsx', sheet_name=userIn)
                print('Done')
    else:
        return

categorize()

What Keyword Would You Like to categorize? Deposit


,Date,Transaction,Amount
0,12/22,DEPOSIT,700.00
1,12/22,ATM CHECK DEPOSIT *****30141501122,
4,01/03,ATM CASH DEPOSIT *****30141501122,
5,01/06,ACH DEPOSIT EXTENSIS VLLC PAYROLL 005535,464.55
65,01/20,ACH DEPOSIT EXTENSIS VLLC PAYROLL 005772,1129.13
70,02/03,ACH DEPOSIT EXTENSIS VLLC PAYROLL 006048,947.31
75,02/17,ACH DEPOSIT EXTENSIS VLLC PAYROLL 006183,1177.42


Would you like to categorize these transactions? (Y/N) y
Enter name of category: asdf
Is this Category Income? (y/n) y
Done
